In [7]:
def convert_rectangle_to_yolo_format(points, img_width, img_height):
    x_min = min(points[0][0], points[1][0], points[2][0], points[3][0])
    y_min = min(points[0][1], points[1][1], points[2][1], points[3][1])
    x_max = max(points[0][0], points[1][0], points[2][0], points[3][0])
    y_max = max(points[0][1], points[1][1], points[2][1], points[3][1])
    
    center_x = (x_min + x_max) / 2.0 / img_width
    center_y = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return (center_x, center_y, width, height)

def convert_point_to_yolo_format(points, img_width, img_height):
    x, y = points[0]
    x = x / img_width
    y = y / img_height
    return (x, y)
def parse_shapes_to_yolo(json_data, img_width, img_height):
    shapes = json_data['shapes']
    rectangle = None
    result_annotations = []
    
    for shape in shapes:
        if shape['shape_type'] == 'rectangle':
            rectangle = convert_rectangle_to_yolo_format(shape['points'], img_width, img_height)
            keypoints = []
        elif shape['shape_type'] == 'point' and rectangle is not None:
            keypoint = convert_point_to_yolo_format(shape['points'], img_width, img_height)
            keypoints.append(keypoint)
            
            # When we find two keypoints following a rectangle, we finalize the YOLO formatted annotation for this group
            if len(keypoints) == 2:
                result_annotations.append((rectangle, keypoints[0],2, keypoints[1], 2))  # Visibility flag is 2
                keypoints = []  # Reset keypoints for the next group
                rectangle = None  # Reset rectangle after processing keypoints

    # Write to a file or process `result_annotations` as needed
    return result_annotations

def write_annotations_to_file(annotations, output_file_path):
    #if not exist create the file
    if not os.path.exists(output_file_path):
        open(output_file_path, 'w').close()
    with open(output_file_path, 'w') as f:
        for annotation in annotations:
            annotation_str = '0 '  # Class ID at the start of the line
            annotation_str += ' '.join(' '.join(map(str, x)) if isinstance(x, tuple) else str(x) for x in annotation)  # Bounding box part
              # Keypoint and visibility flag
            f.write(annotation_str + '\n')


#main funciton
import json
import os

def main():
    input_file_path = 'C:/Users/gbo10/Videos/research/counting_research_algorithms/src/keypoints/GX010080_73_1102.jpg_gamma.json'
    output_file_path = 'C:/Users/gbo10/Videos/research/counting_research_algorithms/src/keypoints/GX010080_73_1102.jpg_gamma.txt'
    img_width = 5312
    img_height = 2988

    with open(input_file_path) as f:
        json_data = json.load(f)

    annotations = parse_shapes_to_yolo(json_data, img_width, img_height)
    write_annotations_to_file(annotations, output_file_path)


main()

In [ ]:

import json
from pathlib import Path


def labelme_to_coco(input_dir, coco_json_path):
    coco_data = {
        "info": {
            "description": "",
            "url": "None",
            "version": "1.0",
            "year": 2024,
            "contributor": "Supervisely User",
            "date_created": "2024-06-15T14:50:58.701Z"
        },
        "licenses": [
            {
                "url": "None",
                "id": 0,
                "name": "None"
            }
        ],
        "images": [],
        "annotations": [],
        "categories": [
            {
                "supercategory": "prawn-model",
                "id": 1,
                "name": "prawn-model",
                "keypoints": [
                    "carapace-start",
                    "eyes"
                ],
                "skeleton": [
                    [
                        1,
                        2
                    ]
                ]
            }
        ]
    }

    image_id = 0
    annotation_id = 0

    for labelme_json_file in Path(input_dir).glob('*.json'):
        with open(labelme_json_file, 'r') as f:
            labelme_data = json.load(f)

        img_width = labelme_data.get("imageWidth", None)
        img_height = labelme_data.get("imageHeight", None)

        if img_width is None or img_height is None:
            print(f"Skipping file {labelme_json_file} due to missing image dimensions.")
            continue

        image_id += 1
        image_info = {
            "license": "None",
            "file_name": labelme_data["imagePath"],
            "url": "None",
            "height": img_height,
            "width": img_width,
            "date_captured": "2024-06-15T14:50:59.895Z",
            "id": image_id
        }
        coco_data["images"].append(image_info)

        current_annotation = None

        for shape in labelme_data['shapes']:
            if shape['shape_type'] == 'rectangle':
                bbox = shape['points']
                x_min = min([p[0] for p in bbox])
                y_min = min([p[1] for p in bbox])
                x_max = max([p[0] for p in bbox])
                y_max = max([p[1] for p in bbox])
                width = x_max - x_min
                height = y_max - y_min

                current_annotation = {
                    "segmentation": [],
                    "area": width * height,
                    "iscrowd": 0,
                    "image_id": image_id,
                    "bbox": [x_min, y_min, width, height],
                    "category_id": 1,
                    "id": annotation_id,
                    "keypoints": [],
                    "num_keypoints": 0
                }
                coco_data["annotations"].append(current_annotation)
                annotation_id += 1

            elif shape['shape_type'] == 'point' and current_annotation is not None:
                point = shape['points'][0]
                current_annotation["keypoints"].extend([point[0], point[1], 2])
                current_annotation["num_keypoints"] += 1

        # Reset current_annotation to ensure new shapes are processed independently
        current_annotation = None

    with open(coco_json_path, 'w') as f:
        json.dump(coco_data, f, indent=4)